In [188]:
import tensorflow as tf

In [189]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train.astype('float32'), x_test.astype('float32')

x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])

x_train, x_test = x_train /255., x_test / 255.

In [190]:
learning_rate_RMSProp = 0.02
learning_rate_GradientDescent = 0.5
num_epochs = 100
batch_size = 256
display_step = 1
input_size = 784
hidden1_size = 128
hidden2_size = 64

In [191]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size)

In [192]:
def random_normal_intializer_with_stddev_1():
  return tf.keras.initializers.RandomNormal(mean=0.0, stddev=1.0, seed=None)

In [193]:
class AutoEncoder(tf.keras.Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()

    self.hidden_layer_1 = tf.keras.layers.Dense(hidden1_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())

    self.hidden_layer_2 = tf.keras.layers.Dense(hidden2_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())

    self.hidden_layer_3 = tf.keras.layers.Dense(hidden1_size,
                                                activation='sigmoid',
                                                kernel_initializer=random_normal_intializer_with_stddev_1(),
                                                bias_initializer=random_normal_intializer_with_stddev_1())

    self.output_layer = tf.keras.layers.Dense(input_size,
                                              activation='sigmoid',
                                              kernel_initializer=random_normal_intializer_with_stddev_1(),
                                              bias_initializer=random_normal_intializer_with_stddev_1())

  def call(self,x):
    H1_output = self.hidden_layer_1(x)
    H2_output = self.hidden_layer_2(H1_output)
    H3_output = self.hidden_layer_3(H2_output)
    X_reconstructed = self.output_layer(H3_output)

    return X_reconstructed, H2_output


In [194]:
class SoftmaxClassifier(tf.keras.Model):
  def __init__(self):
    super(SoftmaxClassifier, self).__init__()

    self.softmax_layer = tf.keras.layers.Dense(10,
                                              activation='softmax',
                                              kernel_initializer='zeros',
                                              bias_initializer='zeros')

  def call(self, x):
    y_pred = self.softmax_layer(x)

    return y_pred

In [195]:
@tf.function
def pretraining_mse_loss(y_pred, y_true):
  return tf.reduce_mean(tf.pow(y_true - y_pred, 2))

In [196]:
@tf.function
def finetuning_cross_entropy_loss(y_pred_softmax, y):
  return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_pred_softmax), axis=[1]))

In [197]:
pretraining_optimizer = tf.optimizers.RMSprop(learning_rate_RMSProp, epsilon=1e-10)

In [198]:
@tf.function
def pretraining_train_step(autoencoder_model, x):

  y_true = x
  with tf.GradientTape() as tape:
    y_pred, _ = autoencoder_model(x)
    pretraining_loss = pretraining_mse_loss(y_pred, y_true)
  gradients = tape.gradient(pretraining_loss, autoencoder_model.trainable_variables)
  pretraining_optimizer.apply_gradients(zip(gradients, autoencoder_model.trainable_variables))

In [199]:
finetuning_optimizer = tf.optimizers.SGD(learning_rate_GradientDescent)

In [200]:
@tf.function
def finetuning_train_step(autoencoder_model, softmax_classifier_model, x, y):
  with tf.GradientTape() as tape:
    y_pred, extracted_features = autoencoder_model(x)
    y_pred_softmax = softmax_classifier_model(extracted_features)
    finetuning_loss = finetuning_cross_entropy_loss(y_pred_softmax, y)
  autoencoder_encoding_variables = autoencoder_model.hidden_layer_1.trainable_variables + autoencoder_model.hidden_layer_2.trainable_variables
  gradients = tape.gradient(finetuning_loss, autoencoder_encoding_variables + softmax_classifier_model.trainable_variables)
  finetuning_optimizer.apply_gradients(zip(gradients, autoencoder_encoding_variables + softmax_classifier_model.trainable_variables))

In [201]:
@tf.function
def compute_accuracy(y_pred_softmax, y):
  correct_prediction = tf.equal(tf.argmax(y_pred_softmax,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

In [203]:
AutoEncoder_model = AutoEncoder()
SoftmaxClassifier_model = SoftmaxClassifier()

In [204]:
for epoch in range(num_epochs):
  for batch_x, _ in train_data:
    _, pretraining_loss_print = pretraining_train_step(AutoEncoder_model, batch_x), pretraining_mse_loss(AutoEncoder_model(batch_x)[0], batch_x)
  if epoch % display_step == 0:
    print("반복(Epoch): %d, Pre-Training 손실 함수(pretraining_loss): %f" % ((epoch + 1), pretraining_loss_print))
print("Step 1 : MNIST 데이터 재구축을 위한 오토인코더 최적화 완료(Pre-Training)")

반복(Epoch): 1, Pre-Training 손실 함수(pretraining_loss): 0.111857
반복(Epoch): 2, Pre-Training 손실 함수(pretraining_loss): 0.093876
반복(Epoch): 3, Pre-Training 손실 함수(pretraining_loss): 0.081634
반복(Epoch): 4, Pre-Training 손실 함수(pretraining_loss): 0.077810
반복(Epoch): 5, Pre-Training 손실 함수(pretraining_loss): 0.067766
반복(Epoch): 6, Pre-Training 손실 함수(pretraining_loss): 0.063725
반복(Epoch): 7, Pre-Training 손실 함수(pretraining_loss): 0.063137
반복(Epoch): 8, Pre-Training 손실 함수(pretraining_loss): 0.058478
반복(Epoch): 9, Pre-Training 손실 함수(pretraining_loss): 0.055381
반복(Epoch): 10, Pre-Training 손실 함수(pretraining_loss): 0.053382
반복(Epoch): 11, Pre-Training 손실 함수(pretraining_loss): 0.052111
반복(Epoch): 12, Pre-Training 손실 함수(pretraining_loss): 0.051626
반복(Epoch): 13, Pre-Training 손실 함수(pretraining_loss): 0.047675
반복(Epoch): 14, Pre-Training 손실 함수(pretraining_loss): 0.047547
반복(Epoch): 15, Pre-Training 손실 함수(pretraining_loss): 0.043246
반복(Epoch): 16, Pre-Training 손실 함수(pretraining_loss): 0.045480
반복(Epoch): 17, Pr

In [205]:
for epoch in range(num_epochs + 100):

  for batch_x, batch_y in train_data:
    batch_y = tf.one_hot(batch_y, depth=10)
    _, finetuning_loss_print = finetuning_train_step(AutoEncoder_model, SoftmaxClassifier_model, batch_x, batch_y), finetuning_cross_entropy_loss(SoftmaxClassifier_model(AutoEncoder_model(batch_x)[1]), batch_y)

  if epoch % display_step == 0:
    print("반복(Epoch): %d, Fine-tuning 손실 함수(finetuning_loss): %f" % ((epoch + 1), finetuning_loss_print))
print("Step 2 : MNIST 데이터 분류를 위한 오토인코더+Softmax 분류기 최적화 완료(Fine-Tuning)")

반복(Epoch): 1, Fine-tuning 손실 함수(finetuning_loss): 0.522633
반복(Epoch): 2, Fine-tuning 손실 함수(finetuning_loss): 0.291853
반복(Epoch): 3, Fine-tuning 손실 함수(finetuning_loss): 0.250015
반복(Epoch): 4, Fine-tuning 손실 함수(finetuning_loss): 0.160413
반복(Epoch): 5, Fine-tuning 손실 함수(finetuning_loss): 0.164048
반복(Epoch): 6, Fine-tuning 손실 함수(finetuning_loss): 0.404381
반복(Epoch): 7, Fine-tuning 손실 함수(finetuning_loss): 0.148457
반복(Epoch): 8, Fine-tuning 손실 함수(finetuning_loss): 0.158081
반복(Epoch): 9, Fine-tuning 손실 함수(finetuning_loss): 0.212685
반복(Epoch): 10, Fine-tuning 손실 함수(finetuning_loss): 0.287406
반복(Epoch): 11, Fine-tuning 손실 함수(finetuning_loss): 0.155827
반복(Epoch): 12, Fine-tuning 손실 함수(finetuning_loss): 0.149580
반복(Epoch): 13, Fine-tuning 손실 함수(finetuning_loss): 0.193131
반복(Epoch): 14, Fine-tuning 손실 함수(finetuning_loss): 0.109462
반복(Epoch): 15, Fine-tuning 손실 함수(finetuning_loss): 0.097395
반복(Epoch): 16, Fine-tuning 손실 함수(finetuning_loss): 0.099171
반복(Epoch): 17, Fine-tuning 손실 함수(finetuning_loss)

In [206]:
print("정확도(오토인코더+Softmax 분류기): %f" % compute_accuracy(SoftmaxClassifier_model(AutoEncoder_model(x_test)[1]), tf.one_hot(y_test, depth=10)))

정확도(오토인코더+Softmax 분류기): 0.964800
